# 讀取字典

In [1]:
import pandas as pd
import os

In [2]:
basePath = '/Volumes/backup_128G/z_repository/Yang_data/學測指考英文'
dataPath = 'data_docx'

A_multiple_path = '{0}/{1}/學測 1 Multiple'.format(basePath, dataPath)
A_cloze_path = '{0}/{1}/學測 2 Cloze'.format(basePath, dataPath)
A_reading_path = '{0}/{1}/學測 3 Reading'.format(basePath, dataPath)

B_multiple_path = '{0}/{1}/指考 1 multiple'.format(basePath, dataPath)
B_cloze_path = '{0}/{1}/指考 2 cloze'.format(basePath, dataPath)
B_discourse_path = '{0}/{1}/指考 3 discourse'.format(basePath, dataPath)
B_reading_path = '{0}/{1}/指考 4 reading'.format(basePath, dataPath)

file_dic = 'permantData/7000Words_20190512_v2.xlsx'
to_a_multiple_xlsx = 'output/GSAT_multiple_20190916_v1.xlsx'
to_a_cloze_xlsx = 'output/GSAT_cloze_20190916_v1.xlsx'
to_a_reading_xlsx = 'output/GSAT_reading_20190916_v1.xlsx'

to_b_multiple_xlsx = 'output/Exam_multiple_20190916_v1.xlsx'
to_b_cloze_xlsx = 'output/Exam_cloze_20190916_v1.xlsx'
to_b_discourse_xlsx = 'output/Exam_discourse_20190916_v1.xlsx'
to_b_reading_xlsx = 'output/Exam_reading_20190916_v1.xlsx'

to_summary_doc = 'output/summary_20190916_v1.xlsx'

write_a_multiple_xlsx = '{0}/{1}'.format(basePath, to_a_multiple_xlsx)
write_a_cloze_xlsx = '{0}/{1}'.format(basePath, to_a_cloze_xlsx)
write_a_reading_xlsx = '{0}/{1}'.format(basePath, to_a_reading_xlsx)

write_b_multiple_xlsx = '{0}/{1}'.format(basePath, to_b_multiple_xlsx)
write_b_cloze_xlsx = '{0}/{1}'.format(basePath, to_b_cloze_xlsx)
write_b_discourse_xlsx = '{0}/{1}'.format(basePath, to_b_discourse_xlsx)
write_b_reading_xlsx = '{0}/{1}'.format(basePath, to_b_reading_xlsx)

write_summary_doc = '{0}/{1}'.format(basePath, to_summary_doc)

read_dic = '{0}/{1}'.format(basePath, file_dic)

In [3]:
dicDf = pd.DataFrame()

with pd.ExcelFile(read_dic) as reader:
    # read sheet by sheet
    for sheet in reader.sheet_names:
#         print(sheet)
        sheetDf = pd.read_excel(reader, sheet, header=None)
        sheetDf = sheetDf.fillna(0)

        dicDf = dicDf.append(sheetDf, ignore_index=True)

# change to lowercase
dicDf[0] = dicDf[0].str.lower()
len(dicDf.index)

6813

In [4]:
dicDf.head()

,0,1,2
0,a,art.,1
1,an,art.,1
2,abandon,v.,4
3,abbreviate,v.,6
4,abbreviation,n.,6


# 將英文字 lemmatize（詞形還原）

In [5]:
# 詞性還原 
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
wtlem = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def findWord(word, dicDataF):
    found = dicDataF.loc[dicDataF[0] == word]
    if found.empty == False:
        return found.iloc[0][2]
    return None
    
def returnWord(word, level):
    return {'lemma':word, 'level':level}

def lemmatizer(word, dicDataF):
    lowerWord = word.lower()
    
    found = findWord(lowerWord, dicDataF)
    if found != None:
        return returnWord(word, found)
    
    lemmaWord = wtlem.lemmatize(lowerWord, wordnet.ADJ)
    if lemmaWord != lowerWord:
        found = findWord(lemmaWord, dicDataF)
        if found != None:
            return returnWord(lemmaWord, found)

    lemmaWord = wtlem.lemmatize(lowerWord, wordnet.VERB)
    if lemmaWord != lowerWord:
        found = findWord(lemmaWord, dicDataF)
        if found != None:
            return returnWord(lemmaWord, found)
        
    lemmaWord = wtlem.lemmatize(lowerWord, wordnet.NOUN)
    if lemmaWord != lowerWord:
        found = findWord(lemmaWord, dicDataF)
        if found != None:
            return returnWord(lemmaWord, found)

    lemmaWord = wtlem.lemmatize(lowerWord, wordnet.ADV)
    if lemmaWord != lowerWord:
        found = findWord(lemmaWord, dicDataF)
        if found != None:
            return returnWord(lemmaWord, found)

    return returnWord(word, 9)

# 讀檔；去掉非英文字，回傳 word 字串，包含引用次數

In [6]:
# pip install python-docx 
import docx

def isAlpha(word):
    try:
        return word.encode('ascii').isalpha()
    except UnicodeEncodeError:
        return False
    
def wordCount(wordList):
#     print(wordList)
    wCount = {}
    for word in wordList:
        if word not in wCount:
            wCount[word] = 1
        else:
            wCount[word] += 1
    return wCount
    
def split2Words(txt):
    wordList = []
#     cleanTxt = txt.replace(',', '').replace('.', '')
    splitWords = txt.split()
#     print(splitWords)
#     splitWords = [x for x in splitWords if ' ' not in x]
    for word in splitWords:
#         if '’' in word or '-' in word:
#             wordList.append(word)
#             continue

        if isAlpha(word) == True:
            wordList.append(word)
        elif word != '':
            print(word, ' includes un-alpha characters.')

    return wordCount(wordList)
    
def replaceMultiple(mainString, toBeReplaces, newString):
#     outString = copy.copy(mainString)
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return mainString

# def readTxtFromTable(filename):
#     fullText = []
#     doc = docx.Document(filename)
#     tables = doc.tables
#     for table in tables:
#         for row in table.rows:
#             for cell in row.cells:
#                 for paragraph in cell.paragraphs:
#                     processedPara = replaceMultiple(paragraph.text, ['…', '’', '‘', '-'] , ' ')
# #                     print(processedPara)
#                     fullText.append(processedPara)
# #                     print(paragraph.text)

#     fullText = ' '.join(fullText)
#     return split2Words(fullText)

def readTxt(filename):
    fullText = []
    # #VINCENT#, 20190917, To highlight the high level words
    plainTxt = []
    doc = docx.Document(filename)
    wordCnt = 0
    for paragraph in doc.paragraphs:
#         print(wordCnt, paragraph.text)
        if len(paragraph.text.replace(' ', '')) == 0:
            continue
        wordCnt += len(paragraph.text.split(' '))
        
        # #VINCENT#, 20190917, To highlight the high level words
        plainTxt.append(paragraph.text)
        processedPara = replaceMultiple(paragraph.text, 
                                        ['…', '’', '‘', '\'', '-', '—', '－', '”', '“', '/', 
                                         ';', '!', '.', '?', ',', ':', ':', '(', ')', '–'] , ' ')
#         print(processedPara)
        if len(processedPara) == 0:
            continue
        fullText.append(processedPara)

#     print(wordCnt)
    fullText = ' '.join(fullText)
    # #VINCENT#, 20190917, To highlight the high level words
    return wordCnt, '\r\n'.join(plainTxt), split2Words(fullText)

# 讀檔列表

In [7]:
def processDoc(wordList):
    resWordList = {}
    for word in wordList:
        lowerWord = word.lower()
        worddic = lemmatizer(lowerWord, dicDf)
#         print(word, wordList[word])
        if worddic['lemma'] not in resWordList:
            resWordList[worddic['lemma']] = {'words':word, 
                                             'count':wordList[word], 
                                             'level':worddic['level'], 
                                             'index':'{0}'.format(worddic['lemma'])}
        else:
            resWordList[worddic['lemma']]['words'] += ';{0}'.format(word)
            resWordList[worddic['lemma']]['count'] += wordList[word]
            resWordList[worddic['lemma']]['index'] = '{0}'.format(worddic['lemma'])
            if resWordList[worddic['lemma']]['level'] != worddic['level']:
                print("ERROR!!", word, worddic)

    #     print(word, lemmatizer(word, dicDf))
    return resWordList

# Parse Directory

In [8]:
def parseDir(filedir, outputFile):
    levelColumns = []
    levelNums = []
    levelWords = []
    with pd.ExcelWriter(outputFile) as writer:
        for root, dirs, files in os.walk(filedir):
            for file in sorted(files):
                # '.filename.docx': temporary file in Mac
                # '~': The file could be temporary opened.
                if '.docx' in file and file[0] != '.' and file[0] != '~':
                    sheetName = file.replace('.docx', '')

                    filename = os.path.join(root, file)
                    print(filename)
                    # #VINCENT#, 20190917, To highlight the high level words
                    wordCnt, plainTxt, wordList = readTxt(filename)
#                     print(plainTxt, wordList)
                    processedList = processDoc(wordList)

#                     print(processedList)
                    # output
                    outDf = pd.DataFrame(processedList).T
                    outDf.sort_values(by=['level', 'words'], inplace=True)
#                     print(outDf.head())

                    # list level words
                    levelColumns.append(sheetName)

                    numWords = {'Word Count': wordCnt, 
                                'Vocabulary Count(All)': 0, 
                                'Vocabulary Count(1-6)': 0,
                                'High Level(5-6)': 0}
                    wordWords = {}
                
                    vocCnt_0_9 = 0
                    vocCnt_1_6 = 0
                    highLevelCnt = 0
#                     for level in range(1, 7):
                    for level in [0, 1, 2, 3, 4, 5, 6, 9]:
                        wordDf = outDf.loc[outDf['level'] == level]
            
                        wordBrackets = ''
                        for idx in range(0, wordDf.index.size):
                            word = wordDf.iloc[idx]['index']
                            wordCount = wordDf.iloc[idx]['count']
                            wordBrackets += '{0}({1});'.format(word, wordCount)
                            vocCnt_0_9 += 1
                            if level >= 1 and level <= 6:
                                vocCnt_1_6 += 1
                            if level >= 5 and level <= 6:
                                highLevelCnt += 1
    
                        count = outDf.loc[outDf['level'] == level]['count'].sum()
        
                        numWords[level] = count
                        wordWords[level] = wordBrackets

                    numWords['Vocabulary Count(All)'] = vocCnt_0_9
                    numWords['Vocabulary Count(1-6)'] = vocCnt_1_6
                    numWords['High Level(5-6)'] = highLevelCnt
#                     print(numWords, wordWords)
                    levelNums.append(numWords)
                    levelWords.append(wordWords)
                    
                    # reindex
                    outDf.set_index('index', inplace=True)
                    
                    # write file
                    outDf.to_excel(writer, sheetName)
                    writer.save()

    levelNumDf = pd.DataFrame(levelNums, index=levelColumns)
    levelWordsDf = pd.DataFrame(levelWords, index=levelColumns)
    return levelNumDf, levelWordsDf

In [9]:
# read_test_path = '/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/test'
# write_test_xlsx = '/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/testOutput/test.xlsx'
# # Only for test
# numDf, wordsDf = parseDir(read_test_path, write_test_xlsx)

In [10]:
resMap = []

# 學測 1 Multiple
numDf, wordsDf = parseDir(A_multiple_path, write_a_multiple_xlsx)
resObj = {'name':'學測 1 Multiple Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'學測 1 Multiple Words', 'data':wordsDf}
resMap.append(resObj)

# 學測 2 Cloze
numDf, wordsDf = parseDir(A_cloze_path, write_a_cloze_xlsx)
resObj = {'name':'學測 2 Cloze Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'學測 2 Cloze Words', 'data':wordsDf}
resMap.append(resObj)

# 學測 3 Reading
numDf, wordsDf = parseDir(A_reading_path, write_a_reading_xlsx)
resObj = {'name':'學測 3 Reading Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'學測 3 Reading Words', 'data':wordsDf}
resMap.append(resObj)

# 指考 1 multiple
numDf, wordsDf = parseDir(B_multiple_path, write_b_multiple_xlsx)
resObj = {'name':'指考 1 multiple Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'指考 1 multiple Words', 'data':wordsDf}
resMap.append(resObj)

# 指考 2 cloze
numDf, wordsDf = parseDir(B_cloze_path, write_b_cloze_xlsx)
resObj = {'name':'指考 2 cloze Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'指考 2 cloze Words', 'data':wordsDf}
resMap.append(resObj)

# 指考 3 discourse
numDf, wordsDf = parseDir(B_discourse_path, write_b_discourse_xlsx)
resObj = {'name':'指考 3 discourse Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'指考 3 discourse Words', 'data':wordsDf}
resMap.append(resObj)

# 指考 4 reading
numDf, wordsDf = parseDir(B_reading_path, write_b_reading_xlsx)
resObj = {'name':'指考 4 reading Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'指考 4 reading Words', 'data':wordsDf}
resMap.append(resObj)


/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-099-Q16to20-M清.docx
88%  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-099-Q21to25- M清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-099-Q26to30- M清.docx
2  includes un-alpha characters.
5  includes un-alpha characters.
2006  includes un-alpha characters.
90%  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-100-Q16to20- M清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-100-Q21to25- M清.docx
23  includes un-alpha characters.
1999  includes un-alpha characters.
30  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-100-Q26to30- M清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-101-Q16to20- M清.docx
7  includes un-alpha characters.
0  inc

/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-099- Q45to48-R清.docx
1890s  includes un-alpha characters.
1920s  includes un-alpha characters.
1930s  includes un-alpha characters.
1960s  includes un-alpha characters.
1980s  includes un-alpha characters.
1990s  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-099- Q49to52-R清.docx
1953  includes un-alpha characters.
1992  includes un-alpha characters.
28  includes un-alpha characters.
000  includes un-alpha characters.
30%  includes un-alpha characters.
US$100  includes un-alpha characters.
600  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-099- Q53to56-R清.docx
10  includes un-alpha characters.
000  includes un-alpha characters.
18  includes un-alpha characters.
12  includes un-alpha characters.
$9  includes un-alpha characters.
10  includes un-alpha characters.
000  includes un-alpha characters

/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-108- Q41to44-R清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-108- Q45to48-R清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-108- Q49to52-R清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-108- Q53to56-R清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 1 multiple/B-099-Q11to15- M 清.docx
20  includes un-alpha characters.
000  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 1 multiple/B-099-Q16to20- M 清.docx
15  includes un-alpha characters.
2008  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 1 multiple/B-100-Q11to15- M 清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 1 multiple/B-100-Q16to20- M 清.docx
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 1 multiple/B-

/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-099- Q40to43- R 清.docx
18  includes un-alpha characters.
24  includes un-alpha characters.
10  includes un-alpha characters.
15  includes un-alpha characters.
24  includes un-alpha characters.
2007  includes un-alpha characters.
20  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-099- Q44to47- R 清.docx
$50  includes un-alpha characters.
500  includes un-alpha characters.
44  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-099- Q48to51- R 清.docx
1999  includes un-alpha characters.
1996  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-100- Q36to39- R 清.docx
US$250  includes un-alpha characters.
US$450  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-100- Q40to43- R 清.docx
50  includes 

/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-106- Q48to51- R 清.docx
16th  includes un-alpha characters.
1740s  includes un-alpha characters.
18th  includes un-alpha characters.
19th  includes un-alpha characters.
1860s  includes un-alpha characters.
Ōyama  includes un-alpha characters.
19th  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-107- Q36to39- R 清.docx
1785  includes un-alpha characters.
19th  includes un-alpha characters.
12  includes un-alpha characters.
1817  includes un-alpha characters.
13  includes un-alpha characters.
60  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-107- Q40to43- R 清.docx
2011  includes un-alpha characters.
224  includes un-alpha characters.
14  includes un-alpha characters.
14  includes un-alpha characters.
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-107- Q44to47- R 清.docx


# Write level words/nums

In [11]:
with pd.ExcelWriter(write_summary_doc) as writer:
    # write file
    for row in resMap:
        name = row['name']
        data = row['data']
        
        data.to_excel(writer, name)
    
    writer.save()